# AFL Model - Part 4 - Weekly Predictions
Now that we have explored different algorithms for modelling, we can implement our chosen model and predict this week's AFL games! All you need to do is run the afl_modelling script each Thursday or Friday to predict the following week's games.

In [17]:
# Import Modules
from afl_feature_creation_v2 import prepare_afl_features
import afl_data_cleaning_v2
import afl_feature_creation_v2
import afl_modelling_v2
import datetime
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

## Creating The Features For This Weekend's Games
To actually predict this weekend's games, we need to create the same features that we have created in the previous tutorials for the games that will be played this weekend. This includes all the rolling averages, efficiency features, elo features etc. So the majority of this tutorial will be using previously defined functions to create features for the following weekend's games.

### Create Next Week's DataFrame
Let's first get our cleaned afl_data dataset, as well as the odds for next weekend and the 2018 fixture.

In [18]:
# Grab the cleaned AFL dataset and the column order
afl_data = afl_data_cleaning_v2.prepare_afl_data()
ordered_cols = afl_data.columns

# Define a function which grabs the odds for each game for the following weekend
def get_next_week_odds(path):
    # Get next week's odds
    next_week_odds = pd.read_csv(path)
    next_week_odds = next_week_odds.rename(columns={"team_1": "home_team", 
                                                "team_2": "away_team", 
                                                "team_1_odds": "odds", 
                                                "team_2_odds": "odds_away"
                                               })
    return next_week_odds

# Import the fixture
# Define a function which gets the fixture and cleans it up
def get_fixture(path):
    # Get the afl fixture
    fixture = pd.read_csv(path)

    # Replace team names and reformat
    fixture = fixture.replace({'Brisbane Lions': 'Brisbane', 'Footscray': 'Western Bulldogs'})
    fixture['Date'] = pd.to_datetime(fixture['Date']).dt.date.astype(str)
    fixture = fixture.rename(columns={"Home.Team": "home_team", "Away.Team": "away_team"})
    return fixture

next_week_odds = get_next_week_odds("data/weekly_odds.csv")
fixture = get_fixture("data/afl_fixture_2018.csv")

In [19]:
fixture.tail()

,Date,Season,Season.Game,Round,home_team,away_team,Venue
215,2018-08-26,2018,216,NaN,St Kilda,North Melbourne,Etihad Stadium
216,2018-09-06,2018,217,NaN,Richmond,Hawthorn,MCG
217,2018-09-07,2018,218,NaN,Melbourne,Geelong,MCG
218,2018-09-08,2018,219,NaN,Sydney,GWS,SCG
219,2018-09-08,2018,220,NaN,West Coast,Collingwood,Optus Stadium


In [20]:
next_week_odds

,home_team,away_team,odds,odds_away
0,Richmond,Hawthorn,1.41,3.40
1,Melbourne,Geelong,1.86,2.18
2,Sydney,GWS,1.83,2.22
3,West Coast,Collingwood,1.66,2.52


Now that we have these DataFrames, we will define a function which combines the fixture and next week's odds to create a single DataFrame for the games over the next 7 days. To use this function we will need Game IDs for next week. So we will create another function which creates Game IDs by using the Game ID from the last game played and adding 1 to it.

In [21]:
# Define a function which creates game IDs for this week's footy games
def create_next_weeks_game_ids(afl_data):
    odds = get_next_week_odds("data/weekly_odds.csv")

    # Get last week's Game ID
    last_afl_data_game = afl_data['game'].iloc[-1]

    # Create Game IDs for next week
    game_ids = [(i+1) + last_afl_data_game for i in range(odds.shape[0])]
    return game_ids


# Define a function which creates this week's footy game DataFrame
def get_next_week_df(afl_data):
    # Get the fixture and the odds for next week's footy games
    fixture = get_fixture("data/afl_fixture_2018.csv")
    next_week_odds = get_next_week_odds("data/weekly_odds.csv")
    next_week_odds['game'] = create_next_weeks_game_ids(afl_data)

    # Get today's date and next week's date and create a DataFrame for next week's games
    todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
    date_in_7_days = (datetime.datetime.today() + datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    fixture = fixture[(fixture['Date'] >= todays_date) & (fixture['Date'] < date_in_7_days)].drop(columns=['Season.Game'])
    next_week_df = pd.merge(fixture, next_week_odds, on=['home_team', 'away_team'])

    # Split the DataFrame onto two rows for each game
    h_df = (next_week_df[['Date', 'game', 'home_team', 'away_team', 'odds', 'Season', 'Round', 'Venue']]
               .rename(columns={'home_team': 'team', 'away_team': 'opponent'})
               .assign(home_game=1))

    a_df = (next_week_df[['Date', 'game', 'home_team', 'away_team', 'odds_away', 'Season', 'Round', 'Venue']]
                .rename(columns={'odds_away': 'odds', 'home_team': 'opponent', 'away_team': 'team'})
                .assign(home_game=0))

    next_week = a_df.append(h_df).sort_values(by='game').rename(columns={
        'Date': 'date',
        'Season': 'season',
        'Round': 'round',
        'Venue': 'venue'
    })
    next_week['date'] = pd.to_datetime(next_week.date)
    next_week['round'] = afl_data['round'].iloc[-1] + 1
    return next_week

In [22]:
next_week_df = get_next_week_df(afl_data)
game_ids_next_round = create_next_weeks_game_ids(afl_data)
next_week_df

,date,round,season,venue,game,home_game,odds,opponent,team
0,2018-09-06,24,2018,MCG,15399,0,3.40,Richmond,Hawthorn
0,2018-09-06,24,2018,MCG,15399,1,1.41,Hawthorn,Richmond
1,2018-09-07,24,2018,MCG,15400,0,2.18,Melbourne,Geelong
1,2018-09-07,24,2018,MCG,15400,1,1.86,Geelong,Melbourne
2,2018-09-08,24,2018,SCG,15401,0,2.22,Sydney,GWS
2,2018-09-08,24,2018,SCG,15401,1,1.83,GWS,Sydney
3,2018-09-08,24,2018,Optus Stadium,15402,0,2.52,West Coast,Collingwood
3,2018-09-08,24,2018,Optus Stadium,15402,1,1.66,Collingwood,West Coast


### Create Each Feature
Now let's append next week's DataFrame to our afl_data, match_results and odds DataFrames and then create all the features we used in the [AFL Feature Creation Tutorial](0.2. afl_feature_creation_tutorial.ipynb). We need to append the games and then feed them into our function so that we can create features for upcoming games.

In [23]:
# Append next week's games to our afl_data DataFrame
afl_data = afl_data.append(next_week_df).reset_index(drop=True)

# Append next week's games to match results (we need to do this for our feature creation to run)
match_results = afl_data_cleaning_v2.get_cleaned_match_results().append(next_week_df)

# Append next week's games to odds
odds = (afl_data_cleaning_v2.get_cleaned_odds().pipe(lambda df: df.append(next_week_df[df.columns]))
       .reset_index(drop=True))

In [24]:
features_df = afl_feature_creation_v2.prepare_afl_features(afl_data=afl_data, match_results=match_results, odds=odds)

In [25]:
features_df.tail()

,game,home_team,away_team,date,round,venue,season,f_odds,f_form_margin_btwn_teams,f_form_past_5_btwn_teams,f_odds_away,f_elo_home,f_elo_away,f_I50_efficiency_home,f_R50_efficiency_home,f_I50_efficiency_away,f_R50_efficiency_away,f_AF_diff,f_B_diff,f_BO_diff,f_CCL_diff,f_CG_diff,f_CL_diff,f_CM_diff,f_CP_diff,f_D_diff,f_ED_diff,f_FA_diff,f_FF_diff,f_G_diff,f_GA_diff,f_HB_diff,f_HO_diff,f_I50_diff,f_ITC_diff,f_K_diff,f_M_diff,f_MG_diff,f_MI5_diff,f_One.Percenters_diff,f_R50_diff,f_SC_diff,f_SCL_diff,f_SI_diff,f_T_diff,f_T5_diff,f_TO_diff,f_UP_diff,f_behinds_diff,f_goals_diff,f_margin_diff,f_opponent_behinds_diff,f_opponent_goals_diff,f_opponent_points_diff,f_points_diff,f_current_odds_prob,f_current_odds_prob_away
1630,15398,St Kilda,North Melbourne,2018-08-26,23,Docklands,2018,5.516150,-3.2,2.0,2.272313,1372.453734,1454.022032,0.730843,0.635819,0.697018,0.654991,27.891262,3.201137,4.754346,-1.881145,-3.924740,-0.528075,-8.045729,-20.584717,36.806235,39.615090,7.018240,-4.709732,-4.535660,-3.372912,23.194704,-18.042370,1.214353,-14.771187,13.611531,11.690647,-109.284521,-0.229945,12.384044,-4.625633,57.158576,1.353070,-1.533659,-6.646259,-3.489492,-15.416140,58.470456,1.223261,-2.257517,-19.923855,0.463481,1.189755,7.602012,-12.321842,0.284269,0.717566
1631,15399,Richmond,Hawthorn,2018-09-06,24,MCG,2018,1.357233,-11.2,3.0,1.777710,1695.224151,1632.526671,0.714151,0.689600,0.692038,0.660748,-200.796649,0.205546,4.650181,-0.618619,15.484721,-9.924090,1.136170,5.649654,-17.290896,-22.271977,3.896107,-6.172837,1.785918,1.340173,7.071065,-28.935188,12.247090,20.655027,-24.361961,-15.920766,1015.686805,3.333539,-2.876146,3.159776,-88.710808,-9.305472,14.037022,-14.553826,-3.492583,5.797341,-17.857837,0.407128,1.177220,6.665371,-1.894502,0.449930,0.805080,7.470451,0.709220,0.294118
1632,15400,Melbourne,Geelong,2018-09-07,24,MCG,2018,1.671054,-24.2,1.0,1.608695,1558.566657,1625.423155,0.660402,0.708517,0.668104,0.759021,71.143537,1.252766,2.925820,0.959649,1.814545,-1.360352,4.151592,10.761703,2.487297,-17.223150,-1.671834,-6.469596,-0.268716,0.797440,-3.674919,25.644698,7.369751,8.626891,6.162216,6.295908,435.479121,3.741955,-11.460528,-7.520927,-55.501462,-2.319974,7.932475,4.321983,-3.341998,3.822951,-13.208704,1.840913,-0.775489,-14.337935,-1.959517,2.247572,11.525916,-2.812019,0.537634,0.458716
1633,15401,Sydney,GWS,2018-09-08,24,SCG,2018,2.154646,-5.8,3.0,2.011444,1645.548991,1597.415860,0.682499,0.756775,0.701884,0.731210,-6.858376,-1.134159,-4.385709,-4.001606,5.329202,-5.472968,1.438888,1.812931,-5.636632,-16.716883,-1.043455,4.695356,-2.118061,-2.793135,10.295133,-16.573064,-7.273899,2.437146,-15.931765,5.216760,-681.002959,-0.152429,-12.869997,5.955020,-33.622983,-1.471366,-38.852506,6.821561,-3.406687,11.156469,-8.593571,-1.974043,-0.860314,-6.165420,2.210121,-0.530105,-0.970509,-7.135929,0.546448,0.450450
1634,15402,West Coast,Collingwood,2018-09-08,24,Optus Stadium,2018,2.032520,20.2,3.0,1.740685,1625.871702,1560.370309,0.696750,0.706745,0.700330,0.694586,-85.634097,-1.562483,1.055257,-0.827688,-12.787818,-4.748463,1.482025,-21.803841,-57.649756,-41.443506,2.262408,1.030872,-1.409203,-0.677635,-79.254067,-4.188338,-3.660205,-5.348511,21.604311,25.318926,158.053807,-2.318048,-2.547266,5.826568,-95.674177,-3.920767,-18.229421,-11.983321,-0.761877,-9.806857,-36.203653,-1.057973,-0.444098,-6.947073,0.123237,0.516879,3.224511,-3.722562,0.602410,0.396825


## Create Predictions For the Upcoming Round
Now that we have our features, we can use our model that we created in part 3 to predict the next round. First we need to filter our features_df into a training df and a df with next round's features/matches. Then we can use the model created in the last tutorial to create predictions. For simplicity, I have hardcoded the parameters we used in the last tutorial.

In [26]:
# Get the train df by only taking the games IDs which aren't in the next week df
train_df = features_df[~features_df.game.isin(next_week_df.game)]

# Get the result and merge to the feature_df
match_results = (pd.read_csv("data/afl_match_results.csv")
                    .rename(columns={'Game': 'game'})
                    .assign(result=lambda df: df.apply(lambda row: 1 if row['Home.Points'] > row['Away.Points'] else 0, axis=1)))

train_df = pd.merge(train_df,  match_results[['game', 'result']], on='game')

train_x = train_df.drop(columns=['result'])
train_y = train_df.result

next_round_x = features_df[features_df.game.isin(next_week_df.game)]

In [27]:
# Fit out logistic regression model - note that our predictions come out in the order of [away_team_prob, home_team_prob]

lr_best_params = {'C': 0.01,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'newton-cg',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

feature_cols = [col for col in train_df if col.startswith('f_')]

# Scale features
scaler = StandardScaler()
train_x[feature_cols] = scaler.fit_transform(train_x[feature_cols])
next_round_x[feature_cols] = scaler.transform(next_round_x[feature_cols])

lr = LogisticRegression(**lr_best_params)
lr.fit(train_x[feature_cols], train_y)
prediction_probs = lr.predict_proba(next_round_x[feature_cols])

modelled_home_odds = [1/i[1] for i in prediction_probs]
modelled_away_odds = [1/i[0] for i in prediction_probs]

In [28]:
# Create a predictions df
preds_df = (next_round_x[['date', 'home_team', 'away_team', 'venue', 'game']].copy()
               .assign(modelled_home_odds=modelled_home_odds,
                      modelled_away_odds=modelled_away_odds)
               .pipe(pd.merge, next_week_odds, on=['home_team', 'away_team'])
               .pipe(pd.merge, features_df[['game', 'f_elo_home', 'f_elo_away']], on='game')
               .drop(columns='game')
           )

In [29]:
preds_df

,date,home_team,away_team,venue,modelled_home_odds,modelled_away_odds,odds,odds_away,f_elo_home,f_elo_away
0,2018-09-06,Richmond,Hawthorn,MCG,1.200786,5.980428,1.41,3.40,1695.224151,1632.526671
1,2018-09-07,Melbourne,Geelong,MCG,1.475141,3.104637,1.86,2.18,1558.566657,1625.423155
2,2018-09-08,Sydney,GWS,SCG,2.176085,1.850279,1.83,2.22,1645.548991,1597.415860
3,2018-09-08,West Coast,Collingwood,Optus Stadium,1.857281,2.166478,1.66,2.52,1625.871702,1560.370309


Alternatively, if you want to generate predictions using a script which uses all the above code, just run the following:

In [30]:
afl_modelling_v2.create_predictions()

,date,home_team,away_team,venue,modelled_home_odds,modelled_away_odds,odds,odds_away,f_elo_home,f_elo_away
0,2018-09-06,Richmond,Hawthorn,MCG,1.200786,5.980428,1.41,3.40,1695.224151,1632.526671
1,2018-09-07,Melbourne,Geelong,MCG,1.475141,3.104637,1.86,2.18,1558.566657,1625.423155
2,2018-09-08,Sydney,GWS,SCG,2.176085,1.850279,1.83,2.22,1645.548991,1597.415860
3,2018-09-08,West Coast,Collingwood,Optus Stadium,1.857281,2.166478,1.66,2.52,1625.871702,1560.370309


## Conclusion
Congratulations! You have created AFL predictions for this week. If you are beginner to this, don't be overwhelmed. The process gets easier each time you do it. And it is super rewarding. In future iterations we will update this tutorial to predict actual odds, and then integrate this with Betfair's API so that you can create an automated betting strategy using Machine Learning to create your predictions!